In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch_dct import dct_2d
from PIL import Image
import torchvision.transforms as transforms
import timm

C:\Users\trung\miniconda3\envs\image_forgery\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [29]:
def log_magnitude(f_shift):
    return torch.log(1 + torch.abs(f_shift))

class ImageToFrequency(nn.Module):
    def __init__(self):
        super(ImageToFrequency, self).__init__()

    def forward(self, x):
        x = torch.fft.rfft2(x, dim=(-2, -1))
        x = torch.fft.fftshift(x)
        x = log_magnitude(x)
        return x

In [33]:
image = Image.open(r'D:\image_forgery_detection\data\CASIA1\train\fake\Sp_S_NNN_T_txt0068_txt0068_0068.jpg')
image = transforms.ToTensor()(image)
image_2 = image * 255.0

In [36]:
ImageToFrequency()(image_2).max   ()


tensor(16.1583)

In [16]:
t = torch.rand(10, 10)
rfft2 = torch.fft.rfft2(t)
rfft2.size()
fft2 = torch.fft.fft2(t)
torch.testing.assert_close(fft2[..., :6], rfft2, check_stride=False)

In [1]:
from src.lit_model import LitModel

C:\Users\trung\miniconda3\envs\image_forgery\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
LitModel()

TypeError: LitModel.__init__() missing 12 required positional arguments: 'num_classes', 'd_model', 'requires_grad', 'drop_rate', 'proj_drop_rate', 'attn_drop_rate', 'drop_path_rate', 'growth_rate', 'num_layers', 'learning_rate', 'weight_decay', and 'patience'